In [1]:
%pip install undetected-chromedriver
%pip install selenium

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import json
import os
import time
import undetected_chromedriver as uc
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [3]:
%pip install requests PyPDF2 pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import requests
import PyPDF2
import json
from io import BytesIO

In [8]:
import requests
import PyPDF2
from io import BytesIO


In [11]:
def findJournalContent(pdf_url):
    
    response = requests.get(pdf_url)

    if response.status_code == 200:
        pdf_file = BytesIO(response.content)

        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

        print(text)
    
    else:
        print("Failed to retrieve the PDF. Status code:", response.status_code)

In [14]:
pdf_url = "https://arxiv.org/pdf/2501.11025"  
# content = findJournalContent(pdf_url)

In [31]:
def findJournalContent(pdf_url, pdf_address, file="catalyst_journals.json"):
    driver = webdriver.Chrome()
    driver.get(pdf_url)
    
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "embed"))
        )
        print(f"PDF found: {pdf_url}")

        response = requests.get(pdf_url, timeout=10)
        if response.status_code == 200:
            pdf_file = BytesIO(response.content)
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""

            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()

            try:
                with open(file, "r") as f:
                    data = json.load(f)
            except:
                data = {}


            data[pdf_address] = text

            with open(file, "w") as f:
                json.dump(data, f, indent=4)

            # print(f"PDF text added to {file} under key {pdf_address}")
        else:
            print("Failed to download the PDF. Status code:", response.status_code)

    except TimeoutException:
        print("Unable to find the PDF within 10 seconds.")
    except NoSuchElementException:
        print("PDF element not found on the page.")
    finally:
        driver.quit()



In [35]:
pdf_url = "https://arxiv.org/pdf/2501.11025"  
content = findJournalContent(pdf_url,"2501_11025")

PDF found: https://arxiv.org/pdf/2501.11025


In [23]:
import requests
from bs4 import BeautifulSoup

def find_arxiv_pdf_links_dict(query, pages=1):
    base_url = "https://arxiv.org/search/"
    pdf_links_dict = {}

    for start in range(0, pages * 50, 50):
        url = f"{base_url}?query={query}&searchtype=all&source=header&start={start}"
        print(f"Scraping page: {url}")
        
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch page {start // 50 + 1}. Status Code: {response.status_code}")
            continue
        
 
        soup = BeautifulSoup(response.content, 'html.parser')

        links = soup.find_all('a', href=lambda href: href and href.startswith("https://arxiv.org/pdf/"))
        for link in links:
            href = link['href']
            
            pdf_address = href.split('/')[-1]
            pdf_address=pdf_address.replace('.', '_') 
            pdf_links_dict[pdf_address] = href

    return pdf_links_dict




In [27]:
query = "catalyst"
pdf_links_dict = find_arxiv_pdf_links_dict(query, pages=1)

print(f"{len(pdf_links_dict)} PDF links scraped:")
cnt=0
for key, link in pdf_links_dict.items():
    print(f"{key}: {link}")
    #displaying only first 5 pdfs
    cnt+=1
    if cnt==5:
        break

Scraping page: https://arxiv.org/search/?query=catalyst&searchtype=all&source=header&start=0
50 PDF links scraped:
2501_12283: https://arxiv.org/pdf/2501.12283
2501_11025: https://arxiv.org/pdf/2501.11025
2501_10954: https://arxiv.org/pdf/2501.10954
2501_10706: https://arxiv.org/pdf/2501.10706
2501_10378: https://arxiv.org/pdf/2501.10378
